In [1]:
import Base: product

using DRR
using Plots
using SharedArrays

using Distributed
using BenchmarkTools

In [2]:
SharedArray{Float64}(10)

System max size of single shmem segment(MB) : 4
System max size of all shmem segments(MB) : 16
Requested size(MB) : 0
Please ensure requested size is within system limits.
If not, increase system limits and try again.


LoadError: SystemError: shm_open() failed for /jl026323qCRV8kaI2giaYIx56ul4: Permission denied
Stacktrace:
 [1] [0m[1msystemerror[22m[0m[1m([22m[90mp[39m::[0mString, [90merrno[39m::[0mInt32; [90mextrainfo[39m::[0mNothing[0m[1m)[22m
[90m   @ [39m[90mBase[39m [90m./[39m[90m[4merror.jl:174[24m[39m
 [2] [0m[1m#systemerror#68[22m
[90m   @ [39m[90m./[39m[90m[4merror.jl:173[24m[39m[90m [inlined][39m
 [3] [0m[1msystemerror[22m
[90m   @ [39m[90m./[39m[90m[4merror.jl:173[24m[39m[90m [inlined][39m
 [4] [0m[1m_shm_mmap_array[22m[0m[1m([22m[90mT[39m::[0mType, [90mdims[39m::[0mTuple[90m{Int64}[39m, [90mshm_seg_name[39m::[0mString, [90mmode[39m::[0mUInt16[0m[1m)[22m
[90m   @ [39m[35mSharedArrays[39m [90m/Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/SharedArrays/src/[39m[90m[4mSharedArrays.jl:681[24m[39m
 [5] [0m[1mshm_mmap_array[22m[0m[1m([22m[90mT[39m::[0mType, [90mdims[39

In [ ]:
findz(x, y; a, b, c, d) = (d - a * x - b * y) / c
findz(x::Tuple{Float64,Float64}; a, b, c, d) = findz(x[1], x[2]; a, b, c, d)
append(xy, z) = Vec3(xy..., z)

# Construct the detector array
function make_plane(detector::Detector)
    d = dotprod(detector.center, detector.normal)
    xs = (-detector.height÷2:1:detector.height÷2) * detector.Δx
    ys = (-detector.width÷2:1:detector.width÷2) * detector.Δy
    xys = product(xs, ys) |> collect
    zs = findz.(xys; a=detector.normal.x, b=detector.normal.y, c=detector.normal.z, d=d)
    return append.(xys, zs)
end
get_rays(camera, plane) = [Ray(camera.center, pixel - camera.center) for pixel in plane]

In [ ]:
# Read the volume
volume, ΔX, ΔY, ΔZ = read_dicom("../data/cxr"; pad=true)
grid, pixels = volume2grid(volume, ΔX, ΔY, ΔZ)

# Define the camera
center = Vec3(180., 180., -100)
camera = Camera(center)

# Define the detector plane
center = Vec3(180., 180., 500.)
normal = Vec3(1., -1., -1.)
height, width = 601, 601
Δx, Δy = 2., 2.
detector = Detector(center, normal, height, width, Δx, Δy)

# Generate the DRR
spacing = 0.1

In [ ]:
function raytrace_trilinear(ray, spacing::Float64, grid, pixels)
    pts = trace.(0:spacing:1; ray=ray)
    interpolations = interpolate.(pts; grid, pixels)
    return sum(interpolations) / length(pts)
end

In [ ]:
# Set up the detector plane
plane = make_plane(detector)
projector = get_rays(camera, plane);

In [ ]:
function make_drr1(projector, spacing, grid, pixels)
    drr = Array{Float64}(undef, length(projector))
    for i in 1:length(projector)
        ray = projector[i]
        drr[i] = raytrace_trilinear(ray, spacing, grid, pixels)
    end
    return drr
end
@btime make_drr1(projector, spacing, grid, pixels);

In [ ]:
# Trace rays through the voxel grid (series)
make_drr2(projector, spacing, grid, pixels) = [raytrace_trilinear(ray, spacing, grid, pixels) for ray in projector]
@btime make_drr2(projector, spacing, grid, pixels);

In [ ]:
length(projector)

In [ ]:
function paradrr(projector, spacing, grid, pixels)
    drr = SharedArray{Float64}(1, length(projector))
    @distributed for i in 1:length(projector)
        ray = projector[i]
        drr[i] = raytrace_trilinear(ray, spacing, grid, pixels)
    end
    return drr
end
@btime paradrr(projector, spacing, grid, pixels);

In [ ]:
@btime paradrr(projector, spacing, grid, pixels);

In [ ]:
function harmonic_distr2(n::Int64)
    x = (+) for i in n:-1:1
        1/i # no x assignment here
    end
    return x
end

In [ ]:
harmonic_distr2(1_)